In [13]:
%load_ext autoreload
%autoreload 2

import time

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np
import pickle

from classes.dataset import *
from classes.model import *
from classes.loss import *
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
size = '3m'
X_encoding = 'bin_y'
y_encoding = 'dense_y'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_data = torch.tensor(np.load(f'data/{X_encoding}_{size}.npy'), device=device)
y_data = torch.tensor(np.load(f'data/{y_encoding}_{size}.npy'), device=device)

idx_data = torch.tensor(np.load(f'data/remove_idx_3m.npy'), device=device)

In [5]:
data_size = X_data.shape[0]

train_dataset = SimpleDynamicDataset(
    X_data[:int(0.8 * data_size)],
    idx_data[:int(0.8 * data_size)],
    y_data[:int(0.8 * data_size)]
)
test_dataset = SimpleDynamicDataset(
    X_data[int(0.8 * data_size):],
    idx_data[int(0.8 * data_size):],
    y_data[int(0.8 * data_size):]
)

train_loader, test_loader = DataLoader(train_dataset, batch_size=100, shuffle=True), DataLoader(test_dataset, batch_size=1000)

del X_data, y_data

In [14]:
epochs = 10

train_steps = len(train_loader)
test_steps = len(test_loader)

print_step = 10

model = RecurrentModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.1)
loss_fn = DoubleLoss()

for epoch in range(epochs):
    print(f'\nEPOCH {epoch + 1}:')
    train_loss = 0.

    start = time.time()
    for i, (X, idx, y) in enumerate(train_loader):
        optimizer.zero_grad()

        cell_pred, update_pred = model(X)

        loss = loss_fn(cell_pred, update_pred, idx, y)

        train_loss += loss.item()

        loss.backward()
        optimizer.step()

        if i % print_step == 0 and i != 0:
            end = time.time()
            
            batch_speed = print_step / (end - start)
            progress = int((i / train_steps) * 50)
            bar = "\u2588" * progress + '-' * (50 - progress)
            
            print(f'Training: |{bar}| {2 * progress}% - loss {train_loss / print_step:.2g} - speed {batch_speed:.2f} batch/s', end='\r', flush=True)
            
            last_batch_speed = batch_speed
            last_train_loss = train_loss
            train_loss = 0.

            start = time.time()

    last_bar = "\u2588" * 50
    print(f'Training: |{last_bar}| {100}% - loss {last_train_loss / print_step:.2g} - speed {last_batch_speed:.2f} batch/s')

    test_loss = 0.
    with torch.no_grad(): # evaluate model on test data
        for X, y in test_loader:
            pred = model(X)

            loss = loss_fn(pred, y)
            test_loss += loss.item()
        
        len_adjust = len(f" |{last_bar}| {100}% - ") - 2
        print(f'Validation:{" " * len_adjust}loss {test_loss / test_steps:.2g}')

    optimizer.param_groups[0]['lr'] *= 0.5 # reduce learning rate by 50% every epoch


EPOCH 1:


KeyboardInterrupt: 

In [38]:
model.eval()

one_hot = pickle.load(open('data/encoder.pkl', 'rb'))

for X, idx, y in train_loader:
    break

In [23]:
cell_pred.shape, update_pred.shape

(torch.Size([1, 81]), torch.Size([1, 9]))

In [35]:
idx.shape

torch.Size([100])

In [59]:
i = np.random.randint(0, 100)

cell_pred, update_pred = model(X[i].unsqueeze(0))

print(cell_pred.argmax(), update_pred.argmax())
print(idx[i], y[i])

tensor(20) tensor(8)
tensor(73) tensor(7)


In [17]:
idx = 0 #np.random.randint(0, 10)

print_grid(X[idx]*9-1, data_type='dense')
print(loss_fn(X[idx], y[idx]))

print_grid((model(X[idx])*9-1).detach(), data_type='dense')
print(loss_fn(model(X[idx].reshape(1, -1)), y[idx].reshape(1, -1)))

print_grid(y[idx]*9-1, data_type='dense')
print(loss_fn(torch.tensor(one_hot.transform(y[idx].reshape(-1, 1)+1).toarray())*100, y[idx]))

-------------------------------
| [0 9 9 0]  [0 9 9 9]  [9 0 0 9] | [0 0 0 9]  [9 0 0 0]  [0 9 0 9] | [0 9 0 0]  [0 0 9 0]  [0 0 0 0] |
| [0 9 0 0]  [0 0 9 0]  [0 9 0 9] | [0 9 9 0]  [0 0 9 9]  [0 9 9 9] | [0 0 0 9]  [9 0 0 0]  [9 0 0 9] |
| [0 0 9 9]  [0 0 0 9]  [9 0 0 0] | [0 0 9 0]  [0 9 0 0]  [9 0 0 9] | [0 9 9 9]  [0 9 0 9]  [0 9 9 0] |
-------------------------------
| [0 0 0 9]  [0 0 9 9]  [0 0 9 0] | [9 0 0 9]  [0 9 9 0]  [9 0 0 0] | [0 9 0 9]  [0 9 0 0]  [0 9 9 9] |
| [9 0 0 9]  [9 0 0 0]  [0 9 9 9] | [0 9 0 0]  [0 9 0 9]  [0 0 9 0] | [0 0 9 9]  [0 9 9 0]  [0 0 0 9] |
| [0 9 0 9]  [0 9 9 0]  [0 9 0 0] | [0 0 9 9]  [0 9 9 9]  [0 0 0 9] | [9 0 0 0]  [9 0 0 9]  [0 0 9 0] |
-------------------------------
| [0 0 9 0]  [9 0 0 9]  [0 0 9 9] | [0 9 0 9]  [0 0 0 9]  [0 9 0 0] | [0 9 9 0]  [0 9 9 9]  [9 0 0 0] |
| [9 0 0 0]  [0 9 0 9]  [0 0 0 9] | [0 9 9 9]  [0 0 9 0]  [0 9 9 0] | [9 0 0 9]  [0 0 9 9]  [0 9 0 0] |
| [0 9 9 9]  [0 9 0 0]  [0 9 9 0] | [9 0 0 0]  [9 0 0 9]  [0 0 9 9] | [0

TypeError: DoubleLoss.forward() missing 2 required positional arguments: 'cell_y' and 'update_y'

In [28]:
test_loss = nn.CrossEntropyLoss()

print(test_loss(torch.tensor([[0, 0, 19.]]), torch.tensor([2])))

tensor(0.)


In [18]:
model(X[0].reshape(1, -1)).reshape(81, 10).argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
print_grid(model(X[0].reshape(1, -1)).reshape(81, 10).argmax(dim=1).numpy())

-------------------------------
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 7  0  0 |
-------------------------------
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
-------------------------------
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
| 0  0  0 | 0  0  0 | 0  0  0 |
-------------------------------


In [7]:
model(X[0])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)